In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


################################################################################
## General Stuff
################################################################################
from timeUtils import clock, elapsed
from listUtils import getFlatList
from time import sleep
from pandas import DataFrame, Series
from ioUtils import getFile, saveFile
from searchUtils import findDirs, findNearest, findAll
from fileUtils import getDirBasics, getBaseFilename
from unicodedata import normalize
from fsUtils import moveDir, setDir, mkDir, isDir, removeDir
from pandasUtils import getRowDataByColValue, getRowData
import operator
from glob import glob
from os.path import join
from collections import Counter


################################################################################
## Mp3 Stuff
################################################################################
from mp3id import mp3ID


################################################################################
## Database Stuff
################################################################################
from dbBase import dbBase
from mainDB import mainDB
from multiArtist import multiartist
from matchAlbums import matchAlbums
from masterdb import masterdb


################################################################################
## Music Stuff
################################################################################
from musicUtils import *
from myMusicDBMap import myMusicDBMap
from musicBase import myMusicBase
from matchMyMusic import matchMyMusic
from matchMusicName import myMusicName
from mergeDB import searchForMutualDBEntries, searchForMutualArtistDBEntries


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Python Version
import sys
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-08-21 21:42:09.895131


# My Music Database Map

In [2]:
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()
#mdb.getKnownDBData()
_, _ = clock("Last Run")

Counter({'Discogs': 4010, 'AllMusic': 3967, 'MusicBrainz': 3746, 'LastFM': 2865, 'DatPiff': 536, 'RockCorner': 481, 'AceBootlegs': 176, 'CDandLP': 173, 'RateYourMusic': 56, 'MusicStack': 5, 'MetalStorm': 0})
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 750818 ID -> Name entries
    Found 685802 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: False
    Found 51701 ID -> Name entries
    Found 47999 Name -> ID entries
    Found 220307 Albums
=========================   musicbrainz   =========================
Summary Statistics For DB: MusicBrainz
    Using Known Artists: False
    Found 128583 ID -> Name entries
    Found 108624 Name -> ID entries
    Found 0 Albums
=========================   acebootlegs   =========================
Summary Statistics For DB: AceBootlegs
    Using

In [3]:
maindb = mainDB(mdb=mdb, create=False, debug=False)
dbdata = maindb.dbdata
#maindb.setDBFull() ## Do this to recreate everything
#maindb.setDBKnown()

=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   datpiff   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================


# Find All My Music and What's Known/Unknown

#### Find My Music

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase(debug=False)
mmb.findArtistAlbums()
_, _ = clock("Last Run")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
My Music Base: ['/Volumes/Piggy/Music/Matched']
Current Time is Fri Aug 21, 2020 21:42:32 for Finding All Artist Albums
Current Time is Fri Aug 21, 2020 21:43:08 for Done with Finding All Artist Albums
Process [Done with Finding All Artist Albums] took 35 seconds.
Current Time is Fri Aug 21, 2020 21:43:08 for Last Run


#### Get Artist Status and Find Unknowns

In [ ]:
%load_ext autoreload
%autoreload
mmm = matchMyMusic(mdb)
mmm.getArtistStatus()
mmm.setMusicBase(mmb)
unknownArtists = mmm.getUnknownArtists()

#### If there are Unknown Artists Try To Find a Match

In [19]:

            
                    
%load_ext autoreload
%autoreload
mdb = myMusicDBMap(debug=False)
mdb.getFullDBData()
mmm = matchMyMusic(mdb)
mmm.searchForMutualDBEntries(maxAdds=2000)
#mmm.getArtistStatus()
#mmm.setMusicBase(mmb)
#unknownArtists = mmm.getUnknownArtists()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Counter({'Discogs': 4065, 'AllMusic': 3970, 'MusicBrainz': 3812, 'LastFM': 3047, 'DatPiff': 537, 'CDandLP': 507, 'RockCorner': 482, 'AceBootlegs': 176, 'RateYourMusic': 61, 'MusicStack': 5, 'MetalStorm': 0})
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 750818 ID -> Name entries
    Found 685802 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: AllMusic
    Using Known Artists: False
    Found 51701 ID -> Name entries
    Found 47999 Name -> ID entries
    Found 220307 Albums
=========================   musicbrainz   =========================
Summary Statistics For DB: MusicBrainz
    Using Known Artists: False
    Found 128583 ID -> Name entries
    Found 108624 Name -> ID entries
    Found 0 Albums
=========================   

mdb.add("Omaha", "CDandLP", "59229701662440")
mdb.add("Omen", "CDandLP", "41551790670393")
mdb.add("Oneida", "CDandLP", "38654001917801")
mdb.add("Oneida", "LastFM", "48770029067")
mdb.add("Ophthalamia", "CDandLP", "69294199868541")
mdb.add("Opium Taylor", "CDandLP", "10027857021887")
mdb.add("Oppressor", "CDandLP", "1386289622245")
mdb.add("Orchestral Manoeuvres in the Dark", "CDandLP", "12707218841657")
mdb.add("Orchestral Manoeuvres in the Dark", "LastFM", "77376066503")
mdb.add("Orchestral Manoeuvres in the Dark", "MusicBrainz", "315013166808171616902920426783055037642")
mdb.add("Oresund Space Collective", "LastFM", "90155293464")
mdb.add("Origin", "CDandLP", "49743989893681")
mdb.add("Origin", "LastFM", "48805535206")
mdb.add("Ours", "LastFM", "29773069426")
mdb.add("Out Of Order", "CDandLP", "89683192033348")
mdb.add("OutKast", "CDandLP", "36184058369057")
mdb.add("OutKast", "LastFM", "59876883734")
mdb.add("Owen", "CDandLP", "83685111891099")
mdb.add("Oxiplegatz", "CDandLP", "37

mdb.add("Signalmen", "CDandLP", "49237654402995")
mdb.add("Silkworm", "CDandLP", "28566612625508")
mdb.add("Silkworm", "LastFM", "96465993805")
mdb.add("Simon Joyner", "CDandLP", "81004534151236")
mdb.add("Simon Joyner", "LastFM", "26617460933")
mdb.add("Sirenia", "RateYourMusic", "15868")
mdb.add("Sister Hazel", "LastFM", "50888170082")
mdb.add("Six Feet Under", "CDandLP", "62358632033690")
mdb.add("Six Feet Under", "LastFM", "90173505339")
mdb.add("Skavoovie and the Epitones", "CDandLP", "45749445211128")
mdb.add("Skirt", "CDandLP", "27473686095545")
mdb.add("Skrew", "CDandLP", "3850206889151")
mdb.add("Skullflower", "CDandLP", "13849592134493")
mdb.add("Skullflower", "LastFM", "27519156120")
mdb.add("Sleep Chamber", "CDandLP", "72247860395015")
mdb.add("Sloppy Seconds", "CDandLP", "48918627212041")
mdb.add("Sloppy Seconds", "LastFM", "94326119119")
mdb.add("Slow Gherkin", "CDandLP", "7172117924472")
mdb.add("Smegma", "CDandLP", "98010789738165")
mdb.add("Smegma", "LastFM", "35656661

mdb.add("Troy Ave", "MusicBrainz", "164093243799557782359416268478682881523")
mdb.add("Trumans Water", "CDandLP", "67918199988953")
mdb.add("Trumans Water", "LastFM", "66374486093")
mdb.add("Tupac Shakur", "CDandLP", "69067256580427")
mdb.add("Ty Tabor", "CDandLP", "31716838852159")
mdb.add("Uncle Tupelo", "LastFM", "53712868810")
mdb.add("Van Halen", "CDandLP", "57569291002602")
mdb.add("Vienna Teng", "LastFM", "35306245327")
mdb.add("Vincent Ingala", "AllMusic", "0002567682")
mdb.add("Vonda Shepard", "LastFM", "20704633085")
mdb.add("W.A.S.P.", "RateYourMusic", "2650")
mdb.add("Wheat", "RockCorner", "492741873")
mdb.add("White Reaper", "AllMusic", "0003247637")
mdb.add("Wisdom Of Harry", "CDandLP", "15123486633374")
mdb.add("Workdogs", "CDandLP", "98928567847000")
mdb.add("Yellowman", "LastFM", "17626136352")
mdb.add("Young Pioneers", "CDandLP", "80058909899206")
mdb.add("Yazoo", "AllMusic", "0000958839")
mdb.add("Lana Lane", "MusicBrainz", "196857543303881625347542467038851762488")


In [ ]:
#artist = "A-Mafia"
#DataFrame(mdb.getArtistData(artist))

In [ ]:
if len(unknownArtists) > 0:
    mmm.matchUnknownArtists(ratioCut=0.75)
    
## Manually look for an unknown artist

#artist = "amazarashi"
#print(DataFrame(mdb.getArtistData(artist)))
#retval = mmm.manuallyMatchUnknownArtist(artist, cutoff=0.75)
#retval
#mdb.add("Chance the Rapper", "Discogs", "3268583")
#mdb.add("Chance the Rapper", "AllMusic", "0003115050")
#mdb.add("Chance the Rapper", "DatPiff", "8924645")
#mdb.save()
#mdb.add("Afrodite", "Discogs", "7376398")
#mdb.add("Afrodite", "MusicBrainz", "153306575689393285671340715814524277900")

#mdb.add("A-Mafia", "Discogs", "745541")
#mdb.add("A-Mafia", "MusicBrainz", "158772143565098757152684085080660304727")
#mdb.add("A-Mafia", "LastFM", "95345008108")
#mdb.rmArtistDBKey("A-Mafia", "AllMusic

In [ ]:
#mdb.add("amazarashi", "Discogs", "2188779")
# ===> Don Toliver
mdb.add("Don Toliver", "AllMusic", "0003746917")
mdb.add("Don Toliver", "MusicBrainz", "93812276224775629138172122201044607096")
mdb.add("Don Toliver", "LastFM", "86354120802")
# ===> Gabby Barrett
mdb.add("Gabby Barrett", "AllMusic", "0003747024")
mdb.add("Gabby Barrett", "MusicBrainz", "25924046156371827739098611565623157892")
mdb.add("Gabby Barrett", "LastFM", "68808881917")
# ===> Jack Harlow
mdb.add("Jack Harlow", "LastFM", "57970864999")
mdb.add("Jack Harlow", "AllMusic", "0003766269")
# ===> Jane Siberry
mdb.add("Jane Siberry", "AllMusic", "0000208303")
# ===> Lizzo
mdb.add("Lizzo", "AllMusic", "0003167672")
# ===> Mario Lanza
mdb.add("Mario Lanza", "AllMusic", "0000228714")
# ===> Nat King Cole
mdb.add("Nat King Cole", "AllMusic", "0000317093")
# ===> Noah Cyrus
mdb.add("Noah Cyrus", "AllMusic", "0002817246")
# ===> 21 Savage
mdb.add("21 Savage", "AllMusic", "0003515232")
# ===> Perry Como
mdb.add("Perry Como", "AllMusic", "0000315336")
# ===> Powfu
mdb.add("Powfu", "AllMusic", "0003926880")
# ===> Saint Jhn
mdb.add("Saint Jhn", "AllMusic", "0003561286")
# ===> Sam Hunt
mdb.add("Sam Hunt", "LastFM", "3350775951")
mdb.add("Sam Hunt", "AllMusic", "0002674420")
# ===> Shoreline Mafia
mdb.add("Shoreline Mafia", "AllMusic", "0003738879")
mdb.add("Shoreline Mafia", "MusicBrainz", "204305478481515897878140840955148416546")
mdb.add("Shoreline Mafia", "LastFM", "70966663108")
# ===> YoungBoy Never Broke Again
mdb.add("YoungBoy Never Broke Again", "AllMusic", "0003582375")
mdb.save()
#unknownArtists.keys()

# Try Matching Unmatched Albums

In [ ]:
dR = 0.2
rC = 0.7

#for db in ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm']:
for db in ['Discogs', 'MusicBrainz', 'AllMusic', 'LastFM']:
#for db in ['MusicBrainz']:
    for albumType in [1]:
        print("="*140)
        mmm.matchMyMusicAlbums(db=db, albumType=albumType, ratioCut=rC, maxCut=rC+dR)
        print("\n\n")

In [ ]:
mmm.mmn.moveMyMatchedMusicAlbums(show=True)

In [ ]:
mmm.mmn.moveMyMatchedMusicAlbums(show=False)

***
***

# Match Albums From My Matched Artists

In [ ]:
resultD = {"ID": None, "Matches": 0, "Score": 0.0, "Best": None}

for artistName,artistDBIDs in artistNameDBIDs.items():
    print(artistName,artistDBIDs)
    for dbArtistID in artistDBIDs:
        print('\t',dbArtistID)
        dbArtistIDAlbums = mdb.getArtistAlbumsFromID(db, dbArtistID, flatten=True)
        print("\t\t",dbArtistIDAlbums)

        
        ### Match Albums (if possible)
        ma = matchAlbums()
        ma.match(unMatchedAlbums, dbArtistIDAlbums)
        if debug:
            print("\t\t{0: <45}{1}\t{2}\t{3}\t{4}".format(dbArtistID, len(dbArtistIDAlbums), ma.near, ma.score, ma.maxval))
        if ma.near < resultD["Matches"]:
            continue
        if ma.score < max([resultD["Score"], cutoff]):
            continue
        resultD = {"ID": dbArtistID, "Matches": ma.near, "Score": ma.score, "Best": ma}
        if debug:
            print("\t\t{0: <45}{1}\t{2}\t{3}\t{4} <-- Match".format(dbArtistID, len(artistAlbums), ma.near, ma.score, ma.maxval))




In [ ]:
dR = 0.1
rC = 0.9

mmm.matchMyMusicAlbums(db="AllMusic", albumType=1, ratioCut=rC, maxCut=rC+dR)

In [ ]:
mmm.moveMyMatchedMusicAlbums(show=False)

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload

from musicBase import myMusicBase
mmb = myMusicBase()
mmb.findArtistAlbums()

***
***

In [ ]:
DatPiffArtists = maindb.getDBData('DatPiff')['Disc'].getMasterSlimArtistDiscogsDB()['DiscArtist']

In [ ]:
retval = {artist: mdb.getArtistDBIDs(db="DatPiff", artistName=artist, cutoff=0.9) for artist in mmb.getArtists()}
retval = {artist: v for artist, v in retval.items() if len(v) > 0}
saveFile(idata=retval, ifile="datPiffMatch.yaml")

In [ ]:
matches = getFile("datPiffMatch.yaml")
for artist, match in matches.items():
    values = list(match.values())
    if values[0] is None:
        print(artist)
    if len(values[0]) != 1:
        print(artist)
    print(values[0][0])
    mdb.add(artist, "DatPiff", str(values[0][0]))
mdb.save()

In [ ]:
mmm.matchUnknownArtist('Notorious B.I.G.')

# UnMatch Artist

In [ ]:

def unMatchArtist(artistName):
    from os.path import join
    from fsUtils import removeDir, isFile, setFile, removeFile

    for musicDir in getMatchedDirs():
        dirval        = join(musicDir, getPrimeDirectory(artistName), artistName)
        if not isDir(dirval):
            continue
            
        matchedDir    = setDir(dirval, "Match")
        mediaTypeDirs = findDirs(matchedDir)
        for mediaTypeDir in mediaTypeDirs:
            for matchDir in findDirs(mediaTypeDir):
                albumName = getUnMatchedDirName(getDirBasics(matchDir)[-1], mediaTypeDir)

                srcDir = matchDir
                dstDir = setDir(dirval, albumName)
                if isDir(dstDir):
                    i = 0
                    while not isDir(dstDir):
                        dstDir = "{0} [Fix-{1}]".format(setDir(dirval, albumName), i)
                        i += 1

                moveDir(srcDir, dstDir, debug=True)

            if isDir(mediaTypeDir):
                DS_Store = setFile(mediaTypeDir, ".DS_Store")
                if isFile(DS_Store):
                    removeFile(DS_Store, debug=True)
                removeDir(mediaTypeDir, debug=True)

        if isDir(matchedDir):
            DS_Store = setFile(matchedDir, ".DS_Store")
            if isFile(DS_Store):
                removeFile(DS_Store, debug=True)
            removeDir(matchedDir, debug=True)
            

In [ ]:
unMatchArtist("Sweet")

In [ ]:
######################################################################
#### Loop Over Prime Directories
######################################################################
for primeDir in getPrimeDirectories():
    artistPrimeDirMap = getArtistPrimeDirMap(primeDir)

    ######################################################################
    #### Loop Over Artist Name <-> Prime Map Items
    ######################################################################
    for artistName, artistPrimeDirs in artistPrimeDirMap.items():
        unMatchArtist(artistName)


***
***

# Merge DBs After Finding Matches

In [ ]:
retval = searchForMutualDBEntries(mdb, minI=-1, cutoff=0.7, maxR=3000)

In [ ]:
retval

In [ ]:
if len(retval) > 0:
    for artistName,artistResult in retval.items():
        for db,dbResult in artistResult.items():
            mdb.add(artistName, db, dbResult["ID"])
    mdb.save()

In [ ]:
mdb.getArtistData("A-Mafia")

In [ ]:
for unknownArtist in unknownArtists.keys():
    print(unknownArtist)

In [ ]:
db = "AllMusic"
for unknownArtist in unknownArtists.keys():
    artistNameDBIDs = mdb.getArtistDBIDs(unknownArtist, db, cutoff=0.99)
    if len(artistNameDBIDs) == 1:
        mdb.add(unknownArtist, db, artistNameDBIDs[unknownArtist][0])
        #print(unknownArtist,'\t',artistNameDBIDs)
mdb.save()

In [ ]:
artistName = "Sweet"
db = "AllMusic"

In [ ]:
artistName = "Sweet"
db = "AllMusic"
unMatchedAlbums = mmb.getArtistAlbumsByArtist(artistName).getUnmatched()
artistNameDBIDs = mdb.getArtistDBIDs(artistName, db)

In [ ]:
resultD = {"ID": None, "Matches": 0, "Score": 0.0, "Best": None}

for artistName,artistDBIDs in artistNameDBIDs.items():
    print(artistName,artistDBIDs)
    for dbArtistID in artistDBIDs:
        print('\t',dbArtistID)
        dbArtistIDAlbums = mdb.getArtistAlbumsFromID(db, dbArtistID, flatten=True)
        print("\t\t",dbArtistIDAlbums)

        
        ### Match Albums (if possible)
        ma = matchAlbums()
        ma.match(unMatchedAlbums, dbArtistIDAlbums)
        if debug:
            print("\t\t{0: <45}{1}\t{2}\t{3}\t{4}".format(dbArtistID, len(dbArtistIDAlbums), ma.near, ma.score, ma.maxval))
        if ma.near < resultD["Matches"]:
            continue
        if ma.score < max([resultD["Score"], cutoff]):
            continue
        resultD = {"ID": dbArtistID, "Matches": ma.near, "Score": ma.score, "Best": ma}
        if debug:
            print("\t\t{0: <45}{1}\t{2}\t{3}\t{4} <-- Match".format(dbArtistID, len(artistAlbums), ma.near, ma.score, ma.maxval))





In [ ]:
from matchAlbums import matchAlbums

def searchForArtistDBEntries(mdb, artistName):
    mmb.getArtistAlbumsByArtist("Sweet").getUnmatched()
    albums = 


def searchForArtistAlbumsDBEntries(mdb, artistName, albums, dbsToMatch=None, cutoff=0.7, num=10, debug=False):
    retval     = {}
    

    ######################################################################
    #### Set Known Albums
    ######################################################################
    knownArtistAlbums = albums
    
    
    ######################################################################
    #### Loop Over Missing DBs
    ######################################################################
    if dbsToMatch is None:
        dbsToMatch = mdb.getDBs()
    for db in dbsToMatch:        
        artistDBIDs = mdb.getArtistDBIDs(artistName, db, cutoff=cutoff, num=num, debug=num)
        print("{0: <20}".format(db), end="\t")
        if "Full" in debug:
            print("Found {0} possible artists in DB".format(len(artistDBIDs)))
        else:
            print("")
        
        
        ######################################################################
        #### Search For Matches in Possible IDs
        ######################################################################
        resultD = {"ID": None, "Matches": num, "Score": 0.0, "Best": None}
        for dbArtistName, dbArtistIDs in artistDBIDs.items():
            for dbArtistID in dbArtistIDs:
                dbArtistIDAlbums = mdb.getArtistAlbumsFromID(db, dbArtistID, flatten=True)
                                
                ma = matchAlbums()
                ma.match(knownArtistAlbums, dbArtistIDAlbums)
                if "ID" in debug or "Full" in debug:
                    print("\t\t{0: <45}{1}\t{2}\t{3}\t{4}".format(dbArtistID, len(dbArtistIDAlbums), ma.near, ma.score, ma.maxval))
                if ma.near < resultD["Matches"]:
                    continue
                if ma.score < max([resultD["Score"], cutoff]):
                    continue
                resultD = {"ID": dbArtistID, "Matches": ma.near, "Score": ma.score, "Best": ma}
                print("\t\t{0: <45}{1}\t{2}\t{3}\t{4} <-- Match".format(dbArtistID, len(artistAlbums), ma.near, ma.score, ma.maxval))

                
        if resultD["ID"] is not None:
            print("\t\t{0: <45}{1}\t{2} <====================================== Best Match".format(resultD["ID"], resultD["Matches"], resultD["Score"]))
            retval[db] = {'ID': resultD["ID"], 'Name': None}
            if "Full" in debug:
                print("\t\t =====>",retval[db])
        else:
            if "Full" in debug:
                print("\t\t =====> No Match")
            retval[db] = None
            
    return retval



def searchForMutualArtistDBEntries(mdb, artistName, num=2, cutoff=0.8, debug=[None]):
    retval     = {}

    
    ######################################################################
    #### Determine Albums To Match
    ######################################################################
    artistAlbums = []
    dbsToMatch   = []
    dbMatches    = mdb.getArtistDataIDs(artistName)
    knownDBs     = []
    for db,artistID in dbMatches.items():
        if artistID is not None:
            artistAlbums.append(mdb.getArtistAlbumsFromID(db, artistID, flatten=True))
            knownDBs.append(db)
        else:
            dbsToMatch.append(db)
    from listUtils import getFlatList
    knownArtistAlbums   = list(set(getFlatList(artistAlbums)))
    print("Searching for matches:  [{0}] using [{1}] albums collected from [{2}] dbs".format(artistName, len(knownArtistAlbums), len(artistAlbums)))
    print("  Will search for matches in these DBs: {0}".format(dbsToMatch))
    
    
    ######################################################################
    #### Loop Over Missing DBs
    ######################################################################
    retval = searchForArtistAlbumsDBEntries(mdb, artistName, knownArtistAlbums, dbsToMatch, cutoff, num, debug)
    return retval
            
            
def searchForMutualDBEntries(mdb, num=2, cutoff=0.8, debug=[None], minI=-1, maxR=50):
    retval = {}
    nR = 0
    
    musicArtists = mdb.getArtists()
    for i, artistName in enumerate(musicArtists):
        if i <= minI:
            continue
        result = searchForMutualArtistDBEntries(mdb, artistName, num, cutoff, debug)
        for db,dbval in result.items():
            if dbval is not None:
                if retval.get(artistName) is None:
                    retval[artistName] = {}
                retval[artistName][db] = dbval
                nR += 1
                
        if nR > maxR:
            break
                
    print("Found {0} new artist matches after looping over {1} artists".format(len(retval), i))
    return retval

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
from random import random

def r(size=0.1):
    val = size*(random() - 0.5)
    return val

def func(x, a, b):
    return (a + b*np.sin(np.pi*x/24.0))

In [ ]:
days = 7

In [ ]:
x = np.linspace(0,24*days,250*days)
y = [func(t, a=1.0, b=2.0) for t in x]
ygen = y + 0.2 * np.random.normal(size=x.size)

In [ ]:
xdata = x
ydata = ygen
popt, pcov = curve_fit(func, xdata, ydata)
popt

In [ ]:
plt.plot(x,y, 'r--', label='truth')
plt.plot(x,ygen, 'g--', label='data')
plt.xlabel('Hour of Week')
plt.ylabel('Energy Usage [arbitrary units]')
plt.plot(xdata, func(xdata, *popt), 'b-', label='fit: a=%5.2f, b=%5.2f' % tuple(popt))
plt.legend()
plt.show()